# Test 1
Austin Derrow-Pinion

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score

In [2]:
# read training data
df_train = pd.read_csv('./Data/HeartDisease_train.csv')
print(df_train.shape)
df_train.head()

(200, 6)


,age,trestbps,chol,sex,fbs,num
0,46.0,138.0,243.0,0.0,0.0,0.0
1,57.0,150.0,168.0,1.0,0.0,0.0
2,47.0,112.0,204.0,1.0,0.0,0.0
3,54.0,122.0,286.0,1.0,0.0,1.0
4,29.0,130.0,204.0,1.0,0.0,0.0


In [3]:
# read test data
df_test = pd.read_csv('./Data/HeartDisease_test.csv')
print(df_test.shape)
df_test.head()

(97, 6)


,age,trestbps,chol,sex,fbs,num
0,76.0,140.0,197.0,0.0,0.0,0.0
1,64.0,125.0,309.0,1.0,0.0,1.0
2,43.0,150.0,247.0,1.0,0.0,0.0
3,41.0,135.0,203.0,1.0,0.0,0.0
4,55.0,130.0,262.0,1.0,0.0,0.0


In [4]:
# one-hot-encode appropriate columns
col_to_one_hot_encode  = ['sex','fbs','num']
for col in col_to_one_hot_encode:
    df_train[col] = df_train[col].astype('category')
    df_test[col] = df_test[col].astype('category')
df_train = pd.get_dummies(df_train)
df_test  = pd.get_dummies(df_test)

In [5]:
# scale appropriate columns
col_to_scale = ['age','trestbps','chol']
df_train[col_to_scale] = scale(df_train[col_to_scale])
df_test[col_to_scale]  = scale(df_test[col_to_scale])

In [6]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
age,200.0,-3.386180e-16,1.002509,-2.788287,-0.821627,0.161703,0.707998,2.456140
trestbps,200.0,8.071321e-16,1.002509,-2.123597,-0.651825,-0.085759,0.480307,2.744572
chol,200.0,-1.126876e-16,1.002509,-2.243313,-0.636274,-0.147999,0.487232,6.062095
sex_0.0,200.0,2.650000e-01,0.442441,0.000000,0.000000,0.000000,1.000000,1.000000
sex_1.0,200.0,7.350000e-01,0.442441,0.000000,0.000000,1.000000,1.000000,1.000000
fbs_0.0,200.0,8.350000e-01,0.372112,0.000000,1.000000,1.000000,1.000000,1.000000
fbs_1.0,200.0,1.650000e-01,0.372112,0.000000,0.000000,0.000000,0.000000,1.000000
num_0.0,200.0,5.450000e-01,0.499220,0.000000,0.000000,1.000000,1.000000,1.000000
num_1.0,200.0,4.550000e-01,0.499220,0.000000,0.000000,0.000000,1.000000,1.000000


In [7]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
age,97.0,-1.693949e-16,1.005195,-2.229457,-0.635982,0.160756,0.729854,2.437149
trestbps,97.0,4.269208e-16,1.005195,-2.130551,-0.675175,-0.115414,0.444346,3.802907
chol,97.0,2.758389e-16,1.005195,-2.473584,-0.698509,0.007486,0.814338,3.134038
sex_0.0,97.0,4.432990e-01,0.499355,0.000000,0.000000,0.000000,1.000000,1.000000
sex_1.0,97.0,5.567010e-01,0.499355,0.000000,0.000000,1.000000,1.000000,1.000000
fbs_0.0,97.0,8.969072e-01,0.305660,0.000000,1.000000,1.000000,1.000000,1.000000
fbs_1.0,97.0,1.030928e-01,0.305660,0.000000,0.000000,0.000000,0.000000,1.000000
num_0.0,97.0,5.257732e-01,0.501929,0.000000,0.000000,1.000000,1.000000,1.000000
num_1.0,97.0,4.742268e-01,0.501929,0.000000,0.000000,0.000000,1.000000,1.000000


In [8]:
# define input and output values
inputs_train  = df_train.drop(['num_0.0','num_1.0'],axis=1).values
inputs_test   = df_test.drop(['num_0.0','num_1.0'],axis=1).values
outputs_train = df_train[['num_0.0','num_1.0']].values
outputs_test  = df_test[['num_0.0','num_1.0']].values

print('training input shape:',inputs_train.shape)
print('testing input shape:',inputs_test.shape)
print('training output shape:',outputs_train.shape)
print('testing output shape:',outputs_test.shape)

training input shape: (200, 7)
testing input shape: (97, 7)
training output shape: (200, 2)
testing output shape: (97, 2)


In [40]:
# define TensorFlow computation graph

x = tf.constant(inputs_train, dtype='float32', shape=[200, 7])
y = tf.constant(outputs_train, dtype='float32', shape=[200, 2])
x_test = tf.constant(inputs_test, dtype='float32', shape=[97, 7])

W = tf.Variable(tf.truncated_normal([7, 2], stddev=0.01))
b = tf.Variable(tf.truncated_normal([1, 2], stddev=0.01))

logits = tf.matmul(x, W) + b
CE = tf.reduce_mean(tf.reduce_sum(-y * tf.log(tf.nn.softmax(logits)), reduction_indices=[1]))

optimizer = tf.train.AdamOptimizer().minimize(CE)

y_train_pred = tf.nn.softmax(tf.matmul(x, W) + b)
y_test_pred = tf.nn.softmax(tf.matmul(x_test, W) + b)

init = tf.initialize_all_variables()

In [41]:
# create a graph session and initialize
sess = tf.Session()
sess.run(init)

In [42]:
# minimize CE
MAXSTEPS = 1000
for step in range(MAXSTEPS+1):
    (_,ce,outputs_train_pred,outputs_test_pred) = sess.run([optimizer,CE,y_train_pred,y_test_pred])
    if (step % 100) == 0:
        acc_train = accuracy_score(outputs_train.argmax(axis=1),outputs_train_pred.argmax(axis=1))
        acc_test  = accuracy_score(outputs_test.argmax(axis=1),outputs_test_pred.argmax(axis=1))
        print('step = %-5d  cross entropy = %f  train accuracy = %f  test accuracy = %f' % (step,ce,acc_train,acc_test))
print('done!')

step = 0      cross entropy = 0.696132  train accuracy = 0.410000  test accuracy = 0.381443
step = 100    cross entropy = 0.636231  train accuracy = 0.680000  test accuracy = 0.608247
step = 200    cross entropy = 0.607548  train accuracy = 0.695000  test accuracy = 0.577320
step = 300    cross entropy = 0.590887  train accuracy = 0.700000  test accuracy = 0.577320
step = 400    cross entropy = 0.581196  train accuracy = 0.715000  test accuracy = 0.577320
step = 500    cross entropy = 0.575605  train accuracy = 0.705000  test accuracy = 0.577320
step = 600    cross entropy = 0.572412  train accuracy = 0.705000  test accuracy = 0.587629
step = 700    cross entropy = 0.570620  train accuracy = 0.705000  test accuracy = 0.587629
step = 800    cross entropy = 0.569640  train accuracy = 0.705000  test accuracy = 0.597938
step = 900    cross entropy = 0.569122  train accuracy = 0.705000  test accuracy = 0.597938
step = 1000   cross entropy = 0.568860  train accuracy = 0.705000  test accuracy